Q6 - Recursive Riddle

Question: Welcome to the Recursive Riddle!
You are given a dataset containing information about a series of magical quests and the hierarchical structure of tasks within each quest.
Each task may depend on the completion of other tasks. Your task is to use recursive techniques to answer the following questions:

- Determine the total number of tasks in each quest.
- Identify the task that has the most dependencies in each quest.
- Calculate the total time required to complete each quest.
- Find the deepest level of task dependency in each quest.
- List the tasks in each quest in the order they should be completed.

Datasets:

magical_quests: Contains columns (quest_id, task_id, task_name, depends_on, time_required).

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Seed for reproducibility
np.random.seed(202)

# Generate synthetic data
quest_ids = np.arange(1, 6)
task_ids = np.arange(1, 21)
task_names = ['Gather Ingredients', 'Cast Spell', 'Brew Potion', 'Enchant Object', 'Defeat Monster', 'Rescue Ally', 'Find Hidden Path', 'Solve Puzzle', 'Forge Weapon', 'Tame Beast']
depends_on_options = [None] + list(task_ids)

data = []
for quest in quest_ids:
    num_tasks = np.random.randint(5, 10)
    tasks = np.random.choice(task_ids, num_tasks, replace=False)
    for task in tasks:
        depends_on = np.random.choice(depends_on_options, np.random.randint(0, 3)).tolist() if np.random.rand() > 0.3 else None
        time_required = np.random.randint(1, 5) * 10  # Time required in minutes
        data.append([quest, task, np.random.choice(task_names), depends_on, time_required])

# Create DataFrame
magical_quests = pd.DataFrame(data, columns=['quest_id', 'task_id', 'task_name', 'depends_on', 'time_required'])

# Convert lists of single elements to integers and empty lists to None
magical_quests['depends_on'] = magical_quests['depends_on'].apply(lambda x: x if isinstance(x, list) and x else None)

# Display the dataset
magical_quests.head()

In [ ]:
magical_quests[magical_quests['quest_id'] == 1]

In [ ]:
def build_dependency_graph(df):
    graph = defaultdict(list)
    time_needed = {}
    for _, row in df.iterrows():
        graph[row['task_id']] = row['depends_on'] if row['depends_on'] is not None else []
        time_needed[row['task_id']] = row['time_required']
    return graph, time_needed


In [ ]:
# Determine the total number of tasks in each quest
tasks_per_quest = magical_quests.groupby('quest_id')['task_id'].count().reset_index()
tasks_per_quest.columns = ['Quest ID', 'Number of Tasks']
tasks_per_quest

In [ ]:
# Identify the task that has the most dependencies in each quest
def count_dependencies(task_id, graph, visited= None):
    if visited is None:
        visited = set()
    if task_id in visited:
        return 0
    visited.add(task_id)
    if not graph[task_id]:
        return 0
    dependencies = [count_dependencies(dep, graph, visited) for dep in graph[task_id] if dep in graph]
    return 1 + (max(dependencies) if dependencies else 0)

dependencies_count = []
for quest, group in magical_quests.groupby('quest_id'):
    graph, _ = build_dependency_graph(group)
    for task_id in group['task_id']:
        dependencies_count.append([quest, task_id, count_dependencies(task_id, graph)])

dependencies_df = pd.DataFrame(dependencies_count, columns=['quest_id', 'task_id', 'dependency_count'])
most_dependencies_task = dependencies_df.loc[dependencies_df.groupby('quest_id')['dependency_count'].idxmax()]
most_dependencies_task

In [ ]:
# Calculate the total time required to complete each quest
def calculate_total_time(task_id, graph, time_required, visited = None):
    if visited is None:
        visited = set()
    if task_id in visited:
        return 0
    visited.add(task_id)
    if not graph[task_id]:
        return  time_required[task_id]
    total_time = time_required[task_id] + sum(calculate_total_time(dep, graph, time_required, visited) for dep in graph[task_id] if dep in graph)
    return total_time

total_time = []
for quest, group in magical_quests.groupby('quest_id'):
    graph, time_required = build_dependency_graph(group)
    quest_total_time = sum(calculate_total_time(task_id, graph, time_required) for task_id in group['task_id'])
    total_time.append([quest, quest_total_time])

total_time_df = pd.DataFrame(total_time, columns=['quest_id', 'total_time'])
total_time_df


In [ ]:
# Find the deepest level of task dependency in each quest
def find_depth(task_id, graph, visited=None):
    if visited is None:
        visited = set()
    if task_id in visited:
        return 0
    visited.add(task_id)
    if not graph[task_id]:
        return 1
    depths = [find_depth(dep, graph, visited) for dep in graph[task_id] if dep in graph]
    return 1 + (max(depths) if depths else 0)

depths = []
for quest, group in magical_quests.groupby('quest_id'):
    graph, _ = build_dependency_graph(group)
    max_depth = max(find_depth(task_id, graph) for task_id in group['task_id'])
    depths.append([quest, max_depth])

depths_df = pd.DataFrame(depths, columns=['quest_id', 'max_depth'])
depths_df

In [ ]:
# List the tasks in each quest in the order they should be completed
def find_order(task_id, graph, visited=None):
    if visited is None:
        visited = set()
    if task_id in visited:
        return []
    visited.add(task_id)
    order = []
    for dep in graph[task_id]:
        if dep in graph:
            order.extend(find_order(dep, graph, visited))
    return order + [task_id]

task_orders = []
for quest, group in magical_quests.groupby('quest_id'):
    graph, _ = build_dependency_graph(group)
    all_orders = []
    for task_id in group['task_id']:
        all_orders.extend(find_order(task_id, graph))
    unique_orders = list(dict.fromkeys(all_orders))
    for i, t in enumerate(unique_orders):
        task_orders.append([quest, t, i])

task_orders_df = pd.DataFrame(task_orders, columns=['quest_id', 'task_id', 'order']).sort_values(by=['quest_id', 'task_id'])
ordered_tasks = pd.merge(task_orders_df, magical_quests[['quest_id', 'task_id', 'task_name']], on=['quest_id', 'task_id'])
ordered_tasks[['quest_id', 'task_name', 'order']]
